In [6]:
import plotly.graph_objects as go
import pandas as pd
import os
import plotly.express as px
from plotly.subplots import make_subplots


# Read the data
file = "/workspaces/SPL/Middleput/points.xlsx"
season1_excel = "/workspaces/SPL/Middleput/season1.xlsx"
season2_excel = "/workspaces/SPL/Middleput/season2.xlsx"
points = pd.read_excel(file)
season1_sorted = pd.read_excel(season1_excel)
season2_sorted = pd.read_excel(season2_excel)


# Adjusting the color palette
rgb_color_palette = {
    'background-color': 'rgb(255, 255, 255)', 
    'color': 'rgb(0, 0, 139)',  # dark blue
    'alternate-color': 'rgb(217, 89, 98)',
    'line-colors': ['rgb(0, 0, 139)', 'rgb(135, 206, 235)', 'rgb(70, 130, 180)'],  # shades of blue
    'bar-colors': ['rgb(255, 69, 0)', 'rgb(255, 140, 0)', 'rgb(255, 165, 0)'],  # shades of orange/red
    'pie-colors': ['rgb(50, 205, 50)', 'rgb(152, 251, 152)', 'rgb(0, 128, 0)']  # shades of green
}

def generate_player_performance_table(season_points, season, player_name, season_sorted_dict):
    """Generate only the player summary table."""
    player_data_season = season_points[season_points['Season'] == season]
    player_data = player_data_season[player_data_season['Player'] == player_name]
    season_sorted = season_sorted_dict.get(season)

    if season_sorted is None or player_data.empty:
        return None

    player_summary = season_sorted[season_sorted['Player'] == player_name]
    player_summary_df = player_summary.drop(columns=['Player']).T
    player_summary_df.columns = [player_name]

    table_fig = go.Figure(go.Table(
        header=dict(values=['Metrics', player_name],
                    fill_color=rgb_color_palette['background-color'],
                    font_color=rgb_color_palette['color']),
        cells=dict(values=[player_summary_df.index, player_summary_df[player_name]],
                   fill_color=[rgb_color_palette['background-color'], 'white'],
                   font_color=[rgb_color_palette['color'], 'black'])
    ))

    table_fig.update_layout(
        title=f"Summary of {player_name} in Season {season}",
        plot_bgcolor=rgb_color_palette['background-color'],
        paper_bgcolor=rgb_color_palette['background-color'],
        font_color=rgb_color_palette['color'],
        font_size=16,  # Increase the font size for better readability
        margin=dict(t=30, b=20)  # Reduce top and bottom margin to bring table closer to the graph
    )

    return table_fig

def generate_player_performance_graph_updated(season_points, season, player_name, season_sorted_dict):
    """Generate player performance graphs."""
    player_data_season = season_points[season_points['Season'] == season]
    player_data = player_data_season[player_data_season['Player'] == player_name]
    season_sorted = season_sorted_dict.get(season)

    if season_sorted is None or player_data.empty:
        return None

    # Creating the subplots using plotly with adjusted vertical_spacing
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        subplot_titles=('Total Points vs Gameweek', 'Goal Points vs Gameweek', 'Game Outcomes', 'Defensive and Midfield Scores vs Gameweek'),
        vertical_spacing=0.15,
        specs=[[{}], [{}], [{'type': 'domain'}], [{}]]
    )

    # Line plot for Total Points vs Gameweek
    fig.add_trace(go.Scatter(x=player_data['Gameweek'], y=player_data['Total Points'], mode='lines+markers', name='Total Points', line_color=rgb_color_palette['line-colors'][0]), row=1, col=1)
    
    # Bar plot for Goal Points vs Gameweek
    fig.add_trace(go.Bar(x=player_data['Gameweek'], y=player_data['Goal Points'], name='Goal Points', marker_color=rgb_color_palette['bar-colors'][0]), row=2, col=1)

    # Pie chart for Game Outcomes with distinct colors
    game_outcomes = player_data['Game Outcome'].value_counts()
    fig.add_trace(go.Pie(labels=game_outcomes.index, values=game_outcomes, hole=.3, marker_colors=rgb_color_palette['pie-colors']), row=3, col=1)

    # Line plots for Defensive and Midfield Scores vs Gameweek
    fig.add_trace(go.Scatter(x=player_data['Gameweek'], y=player_data['Defensive Score Points'], mode='lines+markers', name='Defensive Score', line_color=rgb_color_palette['line-colors'][1]), row=4, col=1)
    fig.add_trace(go.Scatter(x=player_data['Gameweek'], y=player_data['Midfield Score'], mode='lines+markers', name='Midfield Score', line_color=rgb_color_palette['line-colors'][2]), row=4, col=1)

    # Styling with enhanced readability
    fig.update_layout(
        title=f"Performance of {player_name} in Season {season}",
        hovermode="x",
        plot_bgcolor=rgb_color_palette['background-color'],
        paper_bgcolor=rgb_color_palette['background-color'],
        font_color=rgb_color_palette['color']
    )
    fig.update_xaxes(title_text="Gameweek")
    fig.update_yaxes(title_text="Points", row=1, col=1)
    fig.update_yaxes(title_text="Points", row=2, col=1)
    fig.update_yaxes(title_text="Percentage", row=3, col=1)
    fig.update_yaxes(title_text="Points", row=4, col=1)
    fig.update_yaxes(gridcolor='rgb(200, 200, 200)', zerolinecolor='rgb(200, 200, 200)')  # light grey color for gridlines

    return fig

def save_and_plot_player_graphs_and_tables(season_points, season, season_sorted_dict):
    directory_path = f"/workspaces/SPL/Middleput/player_data/s{season}/player_graphs"
    
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    
    points = season_points[season_points['Season'] == season]
    players = points['Player'].unique()

    for player in players:
        table_fig = generate_player_performance_table(points, season, player, season_sorted_dict)
        graph_fig = generate_player_performance_graph_updated(points, season, player, season_sorted_dict)
        
        if table_fig and graph_fig:
            # Combine graph and table into a single HTML content with lesser gaps
            combined_html_content = '<html><head></head><body style="margin:0; padding:0;">' + table_fig.to_html(full_html=False, include_plotlyjs='cdn').replace('<body>', '').replace('</body>', '') + graph_fig.to_html(full_html=False, include_plotlyjs='cdn').replace('<html>', '').replace('</html>', '').replace('<body>', '').replace('</body>', '') + '</body></html>'
            
            with open(f"{directory_path}/{player}.html", "w") as file:
                file.write(combined_html_content)
    
    return f"Combined graphs and tables saved for {len(players)} players in {directory_path}"

# Creating a dictionary for sorted season data
season_sorted_dict = {
    1: season1_sorted,
    2: season2_sorted
}
# Get figures for plotting
save_and_plot_player_graphs_and_tables(points, 2, season_sorted_dict)


'Combined graphs and tables saved for 9 players in /workspaces/SPL/Middleput/player_data/s2/player_graphs'